In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import datetime
import time
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import PolynomialFeatures
from sys import path
path.append(r'E:\data_mining\project\IJCAI-17-口碑\koubei')

import predict
import base
import shop_classify as sc

import matplotlib
myfont = matplotlib.font_manager.FontProperties(fname=r'C:/Windows/Fonts/msyh.ttf')
matplotlib.use('qt4agg')  
#指定默认字体  
matplotlib.rcParams['font.sans-serif'] = ['SimHei']   
matplotlib.rcParams['font.family']='sans-serif'  
#解决负号'-'显示为方块的问题  
matplotlib.rcParams['axes.unicode_minus'] = False 

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


C:\Program Files\Anaconda3\lib\site-packages\matplotlib\__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [2]:
#读取休假表
calenders = base.ReadCalendarCSV();
#读取user_pay_count
user_pay_counts = base.ReadUserPayCountCSV();

##读取城市名称对照表
city_names = base.ReadCityNameCSV();
#读取开店时间数据
shop_open_dates = base.ReadShopOpenDateCSV();

In [3]:
###读取商家信息
shop_infos = base.ReadProcessShopInfoCSV();

In [4]:
%reload_ext autoreload

In [5]:
train_pay_counts = pd.read_csv("../data/train/fill_train_data_regression.txt", sep="\t", encoding='UTF-8', index_col=0)

In [6]:
train_pay_counts.columns = train_pay_counts.columns.map(lambda str_date:datetime.datetime.strptime(str_date, "%Y-%m-%d"))

In [7]:
X_train_date = pd.date_range(start='2016-9-6', end='2016-9-12',freq='D');
X_train_date = X_train_date.append(pd.date_range(start='2016-9-20', end='2016-9-26',freq='D'))
X_train_date = X_train_date.append(pd.date_range(start='2016-10-11', end='2016-10-17',freq='D'))
X_train_data = train_pay_counts[X_train_date]

In [8]:
y_train_week1_date = pd.date_range(start='2016-10-18', end='2016-10-24',freq='D');
y_train_w1_data = train_pay_counts[y_train_week1_date]

In [9]:
y_train_week2_date = pd.date_range(start='2016-10-25', end='2016-10-31',freq='D');
y_train_w2_data = train_pay_counts[y_train_week2_date]

In [10]:
def calculateSingleHolidayMeanData(id, shop_open_dates, train_pay_counts, calenders, week):
    df = train_pay_counts.loc[id][np.array(range(week * 7, (week + 1) * 7))].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
        
    params = {}
    #params[0] = df['count'].mean()
#   params['std'] = df['count'].std()
#   params['max'] = df['count'].max()
#   params['min'] = df['count'].min()
#   params['median'] = df['count'].median()
    
    for i in range(1, 6):
        if i != 4:
            temp_df = df[df['holiday'] == i];
            params[i] = temp_df['count'].mean()
            
    return params;

In [11]:
def calculateHolidayMeansData(shop_open_dates, train_pay_counts, calenders):
    shop_param_dict = {}
    for id in range(1,2001):
        print(id)
        param_dict = {}
        for week in range(3):
            params = calculateSingleHolidayMeanData(id, shop_open_dates, train_pay_counts, calenders, week)
            param_dict[week] = params

        shop_param_dict[id] = param_dict;

    return shop_param_dict;

In [12]:
shop_mean_dict = calculateHolidayMeansData(shop_open_dates, X_train_data, calenders)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [26]:
X_train_data = train_pay_counts[np.array(range(0 * 7, 3 * 7))]

In [31]:

trend_dict = {}
daytype_mean_dict = {}
for id in range(1, 2001):
    
    df = X_train_data.loc[id].T;
    df = df.to_frame();
    df.columns = ['count']
    df['holiday'] = calenders.loc[df.index.strftime('%Y-%m-%d')]['daytype'].values
   
    daytype_means = {}
    for daytype in range(1, 6):
        if daytype != 4:
            sum = 0
            for w in range(0, 3):
                sum = sum + shop_mean_dict[id][w][daytype]
                daytype_means[daytype] = sum / 3
            
            
    trends = []
    for date in df.index:
        trends.append(df['count'][date] - daytype_means[calenders.loc[date.strftime('%Y-%m-%d')]['daytype']])
                
        
    trend_dict[id] = trends
    daytype_mean_dict[id] = daytype_means

In [32]:
trend_df = pd.DataFrame(trend_dict).T

In [34]:
trend_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
1,27.166667,16.166667,48.166667,21.000000,-10.666667,-16.000000,-14.833333,-11.833333,-28.833333,-14.833333,...,9.333333,-4.000000,-9.833333,19.166667,4.166667,-6.833333,-9.000000,1.333333,20.000000,-27.833333
2,26.333333,47.333333,28.333333,15.666667,7.000000,14.666667,46.333333,4.333333,9.333333,-1.666667,...,-36.000000,5.666667,-5.666667,-38.666667,-38.666667,-38.666667,12.666667,29.000000,-20.333333,-38.666667
3,-15.250000,8.750000,15.750000,4.666667,0.333333,-4.666667,-5.250000,-4.250000,-7.250000,-11.250000,...,-3.666667,-4.666667,21.750000,1.750000,-3.250000,-10.250000,-6.333333,3.333333,9.333333,8.750000
4,-3.916667,28.083333,17.083333,15.666667,30.000000,-1.666667,15.083333,-23.916667,3.083333,4.083333,...,4.000000,0.333333,-6.916667,-11.916667,15.083333,-26.916667,3.666667,-34.000000,1.333333,-8.916667
5,-88.333333,-88.333333,-88.333333,-125.000000,-120.666667,18.333333,-88.333333,57.666667,52.666667,89.666667,...,57.333333,-112.666667,85.666667,3.666667,-1.333333,10.666667,69.000000,63.333333,94.333333,54.666667
6,-18.250000,-21.250000,1.750000,-17.333333,-6.333333,-30.666667,-1.250000,0.750000,-8.250000,-2.250000,...,-19.333333,-8.666667,6.750000,6.750000,14.750000,13.750000,32.666667,25.666667,39.333333,6.750000
7,94.833333,14.833333,8.833333,-17.000000,-25.000000,-65.333333,3.833333,-26.166667,-15.166667,-26.166667,...,5.000000,18.666667,-22.166667,4.833333,-38.166667,-3.166667,8.000000,20.000000,46.666667,3.833333
8,8.333333,28.333333,16.333333,12.666667,12.000000,-14.666667,-1.666667,-8.666667,-5.666667,-13.666667,...,-23.000000,-6.666667,-15.666667,10.333333,-7.666667,-5.666667,-17.333333,11.000000,21.333333,-4.666667
9,50.833333,42.833333,23.833333,43.000000,22.333333,-18.666667,45.833333,-3.166667,-27.166667,-12.166667,...,7.333333,30.333333,-29.166667,-16.166667,-39.166667,-49.166667,-44.000000,-29.666667,-11.666667,12.833333
10,-19.916667,-8.916667,-16.916667,-19.000000,-5.000000,-12.000000,-28.916667,-6.916667,24.083333,-10.916667,...,1.000000,-1.000000,0.083333,28.083333,31.083333,-6.916667,16.000000,4.000000,13.000000,16.083333


In [33]:
daytype_mean_dict

{1: {1: 234.83333333333334, 2: 253.0, 3: 262.6666666666667, 5: 244.0},
 2: {1: 72.66666666666667,
  2: 99.33333333333333,
  3: 132.0,
  5: 115.33333333333333},
 3: {1: 66.25,
  2: 77.33333333333333,
  3: 115.66666666666667,
  5: 87.66666666666667},
 4: {1: 97.91666666666667,
  2: 117.33333333333333,
  3: 199.0,
  5: 198.66666666666666},
 5: {1: 219.33333333333334,
  2: 126.0,
  3: 121.66666666666667,
  5: 116.66666666666667},
 6: {1: 76.25,
  2: 88.33333333333333,
  3: 86.33333333333333,
  5: 97.66666666666667},
 7: {1: 131.16666666666666, 2: 132.0, 3: 163.0, 5: 161.33333333333334},
 8: {1: 70.66666666666667,
  2: 66.33333333333333,
  3: 92.0,
  5: 79.66666666666667},
 9: {1: 241.16666666666666,
  2: 241.0,
  3: 280.6666666666667,
  5: 293.6666666666667},
 10: {1: 79.91666666666667, 2: 73.0, 3: 72.0, 5: 90.0},
 11: {1: 81.08333333333333,
  2: 96.33333333333333,
  3: 61.0,
  5: 53.333333333333336},
 12: {1: 66.91666666666667,
  2: 78.66666666666667,
  3: 101.66666666666667,
  5: 96.6666

In [19]:
#生成训练集第一周训练集

In [35]:
# def initFeature(week_date, y_train_data):
#     shop_params = []
#     for id in range(1, 2001):
#         print(id)
#         for day, date in enumerate(week_date):
#             day_type = calenders.loc[date.strftime('%Y-%m-%d')]['daytype']
#             params = {}
#             params['id'] = id
#             #params['day_type'] = day_type
#             params['day'] = day
#             params['count'] = y_train_data.loc[id][date.strftime('%Y-%m-%d')]
#             params['day_type_mean'] = daytype_mean_dict[id][day_type]
            
#             for w in range(0, 3):
#                 params['day_type_trend_w' + str(w)] = shop_mean_dict[id][w][day_type]
            
#             for i in trend_df.columns:
#                 params['trend_' + str(i)] = trend_df.loc[id][i]


#             shop_params.append(params)
#     return shop_params;

def initFeature(week_date):
    shop_params = []
    for id in range(1, 2001):
        print(id)
#       for day, date in enumerate(week_date):
        params = {}
        params['id'] = id
        #params['day'] = day

        for day_type in range(1, 6):
            if day_type != 4:
                params['day_type_mean' + str(day_type)] = daytype_mean_dict[id][day_type]

#             for w in range(0, 3):
#                 params['day_type_trend_w' + str(w)] = shop_mean_dict[id][w][day_type]

        for i in trend_df.columns:
            params['trend_' + str(i)] = trend_df.loc[id][i]


        shop_params.append(params)
    return shop_params;



In [36]:
#week1
#w1_shop_params = initFeature(y_train_week1_date, y_train_w1_data)
w1_shop_params = initFeature(y_train_week1_date)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [37]:
X_train_w1_df = pd.DataFrame(w1_shop_params)

In [45]:
poly = PolynomialFeatures(2,interaction_only=True,include_bias=False)

In [46]:
# X_train_w1_df = pd.DataFrame(w1_shop_params)
# X_train_w1_df = pd.merge(X_train_w1_df, shop_infos[['score', 'comment_cnt', 'shop_level', 'per_pay']], how='inner', on=None, left_on='id', right_on=None,  
#       left_index=False, right_index=True, sort=True,  
#       suffixes=('_x', '_y'), copy=True, indicator=False)

# y_train_w1_df = X_train_w1_df['count']
# X_train_id_df = X_train_w1_df['id']
# X_train_w1_df = X_train_w1_df.drop(['count', 'id'], axis=1)

# X_train_w1_scaled = preprocessing.scale(X_train_w1_df.values)
# X_train_w1_df = pd.DataFrame(X_train_w1_scaled)

# X_train_w1_df['id'] = X_train_id_df

# X_train_w1_df = pd.merge(X_train_w1_df, shop_infos.drop(['score', 'comment_cnt', 'shop_level', 'per_pay'], axis=1), how='inner', on=None, left_on='id', right_on=None,  
#       left_index=False, right_index=True, sort=True,  
#       suffixes=('_x', '_y'), copy=True, indicator=False)
# #X_train_w1_df = X_train_w1_df.astype(np.int)

# X_train_w1_df = X_train_w1_df.drop(['id'], axis=1)



X_train_w1_df = pd.DataFrame(w1_shop_params)
X_train_w1_df = pd.merge(X_train_w1_df, shop_infos[['score', 'shop_level', 'per_pay']], how='inner', on=None, left_on='id', right_on=None,  
      left_index=False, right_index=True, sort=True,  
      suffixes=('_x', '_y'), copy=True, indicator=False)

X_train_id_df = X_train_w1_df['id']
X_train_w1_df = X_train_w1_df.drop('id', axis=1)

X_train_w1_poly = poly.fit_transform(X_train_w1_df.values)
X_train_w1_scaled = preprocessing.scale(X_train_w1_poly)
X_train_w1_df = pd.DataFrame(X_train_w1_scaled)

X_train_w1_df['id'] = X_train_id_df

X_train_w1_df = pd.merge(X_train_w1_df, shop_infos.drop(['score', 'shop_level', 'per_pay'], axis=1), how='inner', on=None, left_on='id', right_on=None,  
      left_index=False, right_index=True, sort=True,  
      suffixes=('_x', '_y'), copy=True, indicator=False)
#X_train_w1_df = X_train_w1_df.astype(np.int)

X_train_w1_df = X_train_w1_df.drop(['id'], axis=1)

In [47]:
y_train_w1 = y_train_w1_data

In [38]:
X_train_w1_df

,day_type_mean1,day_type_mean2,day_type_mean3,day_type_mean5,id,trend_0,trend_1,trend_10,trend_11,trend_12,...,trend_19,trend_2,trend_20,trend_3,trend_4,trend_5,trend_6,trend_7,trend_8,trend_9
0,234.833333,253.000000,262.666667,244.000000,1,27.166667,16.166667,-12.000000,9.333333,-4.000000,...,20.000000,48.166667,-27.833333,21.000000,-10.666667,-16.000000,-14.833333,-11.833333,-28.833333,-14.833333
1,72.666667,99.333333,132.000000,115.333333,2,26.333333,47.333333,-28.333333,-36.000000,5.666667,...,-20.333333,28.333333,-38.666667,15.666667,7.000000,14.666667,46.333333,4.333333,9.333333,-1.666667
2,66.250000,77.333333,115.666667,87.666667,3,-15.250000,8.750000,1.666667,-3.666667,-4.666667,...,9.333333,15.750000,8.750000,4.666667,0.333333,-4.666667,-5.250000,-4.250000,-7.250000,-11.250000
3,97.916667,117.333333,199.000000,198.666667,4,-3.916667,28.083333,-19.333333,4.000000,0.333333,...,1.333333,17.083333,-8.916667,15.666667,30.000000,-1.666667,15.083333,-23.916667,3.083333,4.083333
4,219.333333,126.000000,121.666667,116.666667,5,-88.333333,-88.333333,56.000000,57.333333,-112.666667,...,94.333333,-88.333333,54.666667,-125.000000,-120.666667,18.333333,-88.333333,57.666667,52.666667,89.666667
5,76.250000,88.333333,86.333333,97.666667,6,-18.250000,-21.250000,-15.333333,-19.333333,-8.666667,...,39.333333,1.750000,6.750000,-17.333333,-6.333333,-30.666667,-1.250000,0.750000,-8.250000,-2.250000
6,131.166667,132.000000,163.000000,161.333333,7,94.833333,14.833333,9.000000,5.000000,18.666667,...,46.666667,8.833333,3.833333,-17.000000,-25.000000,-65.333333,3.833333,-26.166667,-15.166667,-26.166667
7,70.666667,66.333333,92.000000,79.666667,8,8.333333,28.333333,4.666667,-23.000000,-6.666667,...,21.333333,16.333333,-4.666667,12.666667,12.000000,-14.666667,-1.666667,-8.666667,-5.666667,-13.666667
8,241.166667,241.000000,280.666667,293.666667,9,50.833333,42.833333,1.000000,7.333333,30.333333,...,-11.666667,23.833333,12.833333,43.000000,22.333333,-18.666667,45.833333,-3.166667,-27.166667,-12.166667
9,79.916667,73.000000,72.000000,90.000000,10,-19.916667,-8.916667,3.000000,1.000000,-1.000000,...,13.000000,-16.916667,16.083333,-19.000000,-5.000000,-12.000000,-28.916667,-6.916667,24.083333,-10.916667


In [27]:
#week2
w2_shop_params = initFeature(y_train_week2_date, y_train_w2_data)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [33]:
X_train_w2_df = pd.DataFrame(w2_shop_params)
X_train_w2_df = pd.merge(X_train_w2_df, shop_infos[['score', 'comment_cnt', 'shop_level', 'per_pay']], how='inner', on=None, left_on='id', right_on=None,  
      left_index=False, right_index=True, sort=True, suffixes=('_x', '_y'), copy=True, indicator=False)

y_train_w2_df = X_train_w2_df['count']
X_train_id_df = X_train_w2_df['id']
X_train_w2_df = X_train_w2_df.drop(['count', 'id'], axis=1)

X_train_w2_scaled = preprocessing.scale(X_train_w2_df.values)
X_train_w2_df = pd.DataFrame(X_train_w2_scaled)

X_train_w2_df['id'] = X_train_id_df

X_train_w2_df = pd.merge(X_train_w2_df, shop_infos.drop(['score', 'comment_cnt', 'shop_level', 'per_pay'], axis=1), how='inner', on=None, left_on='id', right_on=None,  
      left_index=False, right_index=True, sort=True,  
      suffixes=('_x', '_y'), copy=True, indicator=False)
#X_train_w1_df = X_train_w1_df.astype(np.int)

X_train_w2_df = X_train_w2_df.drop(['id'], axis=1)


In [48]:
X_train_w1_df.to_csv("../data/train/gbdt/X_train_w1.txt", sep='\t', index=True, encoding='UTF-8')
y_train_w1.to_csv("../data/train/gbdt/y_train_w1.txt", sep='\t', header= None, index=True, encoding='UTF-8')

In [35]:
X_train_w2_df.to_csv("../data/train/gbdt/X_train_w2.txt", sep='\t', index=True, encoding='UTF-8')
y_train_w2_df.to_csv("../data/train/gbdt/y_train_w2.txt", sep='\t', index=True, encoding='UTF-8')

In [37]:
X_train_w1_df.shape

(14000, 225)